In [1]:
#This code converts to a multixyz file

import re

# Define a mapping from atom type to element
type_to_element = {
    1: 'Ni',
    2: 'O',
}

# Read the dump file and create a new file with elements
with open('./traaj_convert/dump.lammpstrj', 'r') as infile, open('./traaj_convert/dump.lammpstrj_output.xyz', 'w') as outfile:
    atom_lines = []
    num_atoms = 0
    box_dims = [1.0, 1.0, 1.0]
    in_atoms_section = False
    for line in infile:
        if line.startswith("ITEM: TIMESTEP"):
            # Write previous timestep's data if available
            if atom_lines:
                outfile.write(f"{num_atoms}\n")
                outfile.write("Generated by LAMMPS\n")
                outfile.writelines(atom_lines)
                atom_lines = []
        elif line.startswith("ITEM: NUMBER OF ATOMS"):
            num_atoms = int(next(infile).strip())
        elif line.startswith("ITEM: BOX BOUNDS"):
            # Read the box dimensions
            box_dims[0] = float(next(infile).split()[1])
            box_dims[1] = float(next(infile).split()[1])
            box_dims[2] = float(next(infile).split()[1])
        elif line.startswith("ITEM: ATOMS"):
            in_atoms_section = True
        elif in_atoms_section and re.match(r'^\d+ \d+ ', line):
            parts = line.split()
            atom_id = parts[0]
            atom_type = int(parts[1])
            element = type_to_element.get(atom_type, 'X')  # Default to 'X' if type not found
            xs = float(parts[2]) * box_dims[0]
            ys = float(parts[3]) * box_dims[1]
            zs = float(parts[4]) * box_dims[2]
            atom_lines.append(f'{element} {xs} {ys} {zs}\n')

    # Write any remaining atom lines
    if atom_lines:
        outfile.write(f"{num_atoms}\n")
        outfile.write("Generated by LAMMPS\n")
        outfile.writelines(atom_lines)

In [2]:
#This code converts to a XDATCAR file

from ase.io import read, write

def sort_atoms_by_element(atoms):
    """Sort atoms in an Atoms object by element type."""
    sorted_indices = sorted(range(len(atoms)), key=lambda i: atoms[i].symbol)
    return atoms[sorted_indices]

# Read the multi-frame XYZ file
frames = read('./traaj_convert/dump.lammpstrj_output.xyz', index=':')

# Sort the atoms in each frame
sorted_frames = [sort_atoms_by_element(frame) for frame in frames]

# Box bounds values
box_bounds = [25.02, 17.691811665287418, 50.32022347141766]

# Convert positions to Cartesian coordinates
for frame in sorted_frames:
    frame.set_cell([
        [box_bounds[0], 0.0, 0.0],
        [0.0, box_bounds[1], 0.0],
        [0.0, 0.0, box_bounds[2]]
    ])
    frame.set_pbc([True, True, True])
    frame.positions = frame.get_positions()

# Write the sorted frames to an XDATCAR file
write('./traaj_convert/XDATCAR', sorted_frames)

# Modify the XDATCAR file to replace the specific section
with open('./traaj_convert/XDATCAR', 'r') as file:
    lines = file.readlines()

# Replace "Direct" with "Cartesian" and adjust box bounds
if 'Direct configuration=' in lines:
    start_index = lines.index('Direct\n')
    if start_index is not None:
        lines[start_index] = 'Cartesian\n'
    
# Replace the section with the new box bounds
lines[2] = f"   {box_bounds[0]:.6f} 0.000000 0.000000\n"
lines[3] = f"   0.000000 {box_bounds[1]:.6f} 0.000000\n"
lines[4] = f"   0.000000 0.000000 {box_bounds[2]:.6f}\n"

# Write the modified content back to the XDATCAR file
with open('./traaj_convert/XDATCAR', 'w') as file:
    file.writelines(lines)

In [3]:
import MDAnalysis as mda
import numpy as np
from scipy.spatial.distance import cdist

# Load the topology and trajectory files
#u = mda.Universe('./traaj_convert/topology_output.data', './traaj_convert/dump.lammpstrj_output.xyz', topology_format='DATA', format='XYZ')
u = mda.Universe( './traaj_convert/dump.lammpstrj_output.xyz', format='XYZ')

# Select Ni and O atoms based on their types in the topology
ni_atoms = u.select_atoms('name Ni')
o_atoms = u.select_atoms('name O')

# Initialize a list to store distances
nearest_neighbor_distances = []

# Distance cutoff (you can adjust this value as needed)
distance_cutoff = 2.5  # example cutoff in Angstroms

# Calculate distances for each frame
for ts in u.trajectory:
    # Get positions of Ni and O atoms
    ni_positions = ni_atoms.positions
    o_positions = o_atoms.positions
    
    # Calculate pairwise distances using cdist
    distances = cdist(ni_positions, o_positions)
    
    # Find the minimum distance for each Ni atom to any O atom
    min_distances = np.min(distances, axis=1)
    
    # Filter distances by the cutoff
    nearest_distances = min_distances[min_distances < distance_cutoff]
    
    # Append the nearest distances to the list
    nearest_neighbor_distances.extend(nearest_distances)

# Convert the list of distances to a single numpy array
nearest_neighbor_distances = np.array(nearest_neighbor_distances)

# Calculate the average distance
average_nearest_neighbor_distance = np.mean(nearest_neighbor_distances)

print(f'Average nearest neighbor Ni-O distance: {average_nearest_neighbor_distance:.3f} Å')

Average nearest neighbor Ni-O distance: 2.076 Å
